In [ ]:
import batoid
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
%matplotlib inline

In [ ]:
D = 0.1
m0 = batoid.ConstMedium(1.000277)  # air
w, n = np.genfromtxt(os.path.join(batoid.datadir, "media", "silica_dispersion.txt")).T
w *= 1e-6
m1 = batoid.TableMedium(batoid.Table(w, n, batoid.Table.Interpolant.linear))
wavelength = 500e-9
n0 = m0.getN(wavelength)
n1 = m1.getN(wavelength)
R1 = 0.5  # convex entrance surface
R2 = -0.5  # convex exit surface
d = 0.01
# Lens maker equation:
f = 1./((n1-n0)*(1./R1 - 1./R2 + (n1-n0)*d/R1/R2))

In [ ]:
def makeSystem(focus):
    systemStr = """
    opticalSystem:
      type: CompoundOptic
      inMedium: air
      items:
        -
          type: RefractiveInterface
          surface:
            type: Sphere
            R: {R1}
          obscuration:
            type: ClearCircle
            radius: {Do2}
          inMedium: 1.000277
          outMedium: silica
          name: L1
        -
          type: RefractiveInterface
          surface:
            type: Sphere
            R: {R2}
          obscuration:
            type: ClearCircle
            radius: {Do2}
          coordSys:
            z: {d}
          inMedium: silica
          outMedium: 1.000277
          name: L2
        -
          type: Detector
          surface:
            type: Plane
          obscuration:
            type: ClearCircle
            radius: {Do2}
          coordSys:
            z: {f}
          inMedium: 1.000277          
          name: D
    """.format(**dict(R1=R1, R2=R2, Do2=D/2, d=d, f=f+focus))
    config = yaml.load(systemStr)
    system = batoid.parse.parse_optic(config['opticalSystem'])
    return system

In [ ]:
@interact(theta_x=widgets.FloatSlider(min=-5,max=5,step=1,value=0.0, continuous_update=False),
          theta_y=widgets.FloatSlider(min=-5,max=5,step=1,value=0.0, continuous_update=False),
          wavelength=widgets.FloatSlider(min=0.4,max=0.7,step=0.05,value=0.5, continuous_update=False),
          focus=widgets.FloatSlider(min=-20, max=20, step=1,value=0.0, continuous_update=False))
def spot(theta_x, theta_y, wavelength, focus):
    """Display a spot diagram for a Newtonian telescope.

    @param theta_x  Field angle in degrees
    @param theta_y  Field angle in degrees
    @param wavelength in microns
    @param focus    Defocus distance in mm
    """
    system = makeSystem(focus*1e-3)
    xcos = np.sin(theta_x*np.pi/180)
    ycos = np.sin(theta_y*np.pi/180)
    zcos = np.sqrt(1-xcos**2-ycos**2)
    rays = batoid.circularGrid(0.2, D/2*0.999, 0, xcos, ycos, zcos, 16, 64, wavelength*1e-6, 1.000277)
    
    system.traceInPlace(rays)
    batoid.trimVignettedInPlace(rays)
    plt.scatter(1e3*(rays.x-np.mean(rays.x)), 1e3*(rays.y-np.mean(rays.y)), s=1)
    plt.xlim(-1, 1)
    plt.ylim(-1, 1)
    plt.xlabel("mm")
    plt.ylabel("mm")
    plt.title(r"$\theta_x = {:4.2f}\,,\theta_y = {:4.2f}\,,\lambda = {:4g}\,, f={:4.2f}$".format(theta_x, theta_y, wavelength, focus))
    plt.axes().set_aspect('equal')    